# 数据结构
## Series
### Series的创建

In [38]:
import numpy as np
import pandas as pd

- Series是带标签的一维数组，轴标签统称为索引
- Series的创建由数据部分和index(索引)部分组成
- 调用`s = pd .Series(data, index=index)`即可创建
- data是多维数组的时候，index的长度必须和data的长度一致，如果没有提供索引，那么就会自动补齐数值索引，即len[0, ..., len(data)-1]

In [2]:
s = pd.Series(np.random.randint(5,10,5),index=['a','b','c','d','e'])
s2 = pd.Series(np.random.randn(10))
print('Generate with Index\n',s)
print('Generate without Index\n',s2)

Generate with Index
 a    5
b    7
c    6
d    9
e    5
dtype: int32
Generate without Index
 0    0.159340
1   -0.088752
2    1.308632
3    0.203625
4    1.625742
5   -1.031789
6   -3.060075
7    0.403177
8   -0.076317
9   -1.037448
dtype: float64


- Series可以用字典进行实例化，Series按照创建的index顺序进行排序

In [3]:
d = {'b':1,'c':2}
d = pd.Series(d,index=['c','b'])
print(d)

c    2
b    1
dtype: int64


- 如果创建的Series的date是标量，那么必须提供索引，并且每个索引都重复该标量


In [4]:
pd.Series(5,index=['a','b'])

a    5
b    5
dtype: int64

### Series类似多维数组
- 操作与`ndarray`类似，支持大多数`NumPy`函数，支持索引切片

In [5]:
s[0]

5

In [6]:
s[:3]

a    5
b    7
c    6
dtype: int32

- 中位数函数median()

In [7]:
print(s.median())
s[s > s.median()]

6.0


b    7
d    9
dtype: int32

- 和NumPy数组一样，Series也支持dtype


In [8]:
s.dtype

dtype('int32')

- `Series.array`用于提取Series数组
- 执行不用索引的操作的时候，提取数组比较方便
- Series.array一般是扩展数组。就是将N个`numpy.ndarray`包在一起的打包器。

In [9]:
s.array

<PandasArray>
[5, 7, 6, 9, 5]
Length: 5, dtype: int32

- Series只是类似于多维数组，如果提取**真正**的多维数组，需要使用`Series.to_numpy()`
- 这里返回的是NumPy多维数组

In [10]:
s.to_numpy()

array([5, 7, 6, 9, 5])

### Series类似字典
- 可以用索引标签提取值或设置值
- 如果引用Series里面没有的标签可能会出发异常

In [11]:
s['a']

5

In [12]:
'c' in s

True

- `get`方法可以提取`Series`里面没有的标签，返回None或者指定默认值

In [13]:
s.get('f')

In [14]:
s.get('f',np.nan)

nan

### 矢量操作与对齐Series标签
- Series和NumPy数组一样，不用循环每个值就可以进行矢量运算。
- 而且Series支持大多数NumPy多维数组的方法

In [15]:
s+s

a    10
b    14
c    12
d    18
e    10
dtype: int32

In [16]:
s*2

a    10
b    14
c    12
d    18
e    10
dtype: int32

In [17]:
np.exp(s)

a     148.413159
b    1096.633158
c     403.428793
d    8103.083928
e     148.413159
dtype: float64

- Series和多维数组的主要区别在于，Series之间的操作会自动基于标签对齐数据。因此不用管执行计算操作的Series是否有相同的标签

In [18]:
s[1:]+s[:-1]

a     NaN
b    14.0
c    12.0
d    18.0
e     NaN
dtype: float64

- 操作索引未对齐的Series，计算结果是所有涉及的索引的并集。
如果在Series找不到标签，预算结果就标记为NaN，缺失。
#### 注意
让不同索引对象操作的默认结果生成索引并集，是为了避免信息丢失。就算缺失了数据，标签仍然包含重要的信息。
不过也可以用`dropna`函数清除含有缺失值的标签。

In [19]:
pd.Series.dropna(s[1:]+s[:-1])

b    14.0
c    12.0
d    18.0
dtype: float64

### 名称属性
- Series支持`name`属性

In [20]:
s = pd.Series(np.random.randint(0,5,5), name='something')

In [21]:
s

0    0
1    2
2    0
3    2
4    3
Name: something, dtype: int32

In [22]:
s.name

'something'

- 一般情况下，Series自动分配`name`,特别是提取一维DataFrame切片时。
- `pandas.Series.rename()`用于重命名Series

In [23]:
s2 = s.rename('differnent')

In [24]:
s2.name

'differnent'

- 注意这里s和s2指向不同的对象

## DataFrame
DataFrame是由多种类型的列构成的而二维标签数据结构。一个DataFrame对应一个Excel的sheet。
DataFrame支持多种类型的输入数据
- 一维 ndarray、列表、字典、Series字典
- 二维 numpy.ndarray
- 结构多维数组或记录多维数组
- Series
- DataFrame
除了数据，还可以有选择的传递index（行标签）和columns（列标签）参数。传递了索引或者列，就可以确保生成的DataFrame里包含索引或者列。Series字典加上指定的索引是，会丢弃与传递的索引不匹配的所有数据。
没有传递轴标签的时候，按常规依据输入数据进行构建

### 用Series字典或字典生成DataFrame
生成的**索引**是每个Series索引的并集。先把嵌套字典转换为Series。如果没有指定列，DataFrame的列就是字典键的有序列表。

In [25]:
d = {'one':pd.Series([1.,2.,3.],index=['a','b','c']),
     'two':pd.Series([1.,2.,3.,4.],index=['a','b','c','d'])}

df = pd.DataFrame(d)
print(df)

   one  two
a  1.0  1.0
b  2.0  2.0
c  3.0  3.0
d  NaN  4.0


In [26]:
print('Index',df.index)
print('column',df.columns)

Index Index(['a', 'b', 'c', 'd'], dtype='object')
column Index(['one', 'two'], dtype='object')


- 指定列与数据字典一起传递的时候，传递的列会覆盖字典的键

In [27]:
pd.DataFrame(d, index=['b','c','a'])

,one,two
b,2.0,2.0
c,3.0,3.0
a,1.0,1.0


In [28]:
pd.DataFrame(d, index=['d','b','a'], columns=['two','three'])

,two,three
d,4.0,NaN
b,2.0,NaN
a,1.0,NaN


### 用多维数组字典、列表生成DataFrame
多维数组长度必须相同。如果传递了索引参数，index的长度必须与数组一直。如果没有传递索引参数，生成的结果是`range(n)`,`n`是数组长度

In [29]:
d = {'one':[1.,2.,3.,4.],
     'two':[4.,3.,2.,1.,]}

In [30]:
pd.DataFrame(d)

,one,two
0,1.0,4.0
1,2.0,3.0
2,3.0,2.0
3,4.0,1.0


In [31]:
index = ['a','b','c','d']
pd.DataFrame(d, index=index)

,one,two
a,1.0,4.0
b,2.0,3.0
c,3.0,2.0
d,4.0,1.0


### 备选构建器
`DataFrame.from_dict`接收字典组成的字典或数组序列字典，生产呢个DataFrame。除了`orient`参数默认为`columns`,本构建器的操作与`DataFrame`类似。把`orient`参数设置为`index`，即可以把字典的键作为行标签

#### DataFrame.from_records

In [53]:
data = np.array([(1, 2., b'Hello'), (2, 3., b'World')],
      dtype=[('A', '<i4'), ('B', '<f4'), ('C', 'S10')])
data

array([(1, 2., b'Hello'), (2, 3., b'World')],
      dtype=[('A', '<i4'), ('B', '<f4'), ('C', 'S10')])

In [49]:
pd.DataFrame.from_dict(dict([('A',[1,2,3]),('B',[4,5,6])]))

,A,B
0,1,4
1,2,5
2,3,6


`orient='index'`的时候，键是行标签。这里还传递了列标签

In [48]:
pd.DataFrame.from_dict(dict([('A',[1,2,3]),('B',[4,5,6])]),orient='index',columns=['one','two','three'])

,one,two,three
A,1,2,3
B,4,5,6


### 提取、添加、删除列
DataFrame可以理解为带索引的Series字典，提取、设置、删除列的操作与字典类似

In [32]:
df['one']

a    1.0
b    2.0
c    3.0
d    NaN
Name: one, dtype: float64

- 添加列

In [33]:
df['three'] = df['one'] * df['two']
df['flag'] = df['one'] > 2

In [34]:
df

,one,two,three,flag
a,1.0,1.0,1.0,False
b,2.0,2.0,4.0,False
c,3.0,3.0,9.0,True
d,NaN,4.0,NaN,False


- 删除（del、pop）列的方式也与字典类似

In [35]:
del df['two']
three = df.pop('three')

In [36]:
df

,one,flag
a,1.0,False
b,2.0,False
c,3.0,True
d,NaN,False


- 标量值以广播的方式填充列

In [39]:
df['foo'] = 'bar'
df

,one,flag,foo
a,1.0,False,bar
b,2.0,False,bar
c,3.0,True,bar
d,NaN,False,bar


- 插入与DataFrame索引不同的Series

array([(1, 2., b'Hello'), (2, 3., b'World')],
      dtype=[('A', '<i4'), ('B', '<f4'), ('C', 'S10')])